In [ ]:
!pip install -U pip
%pip install -Uq "unstructured[all-docs]" Pillow lxml
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community langchain-openai
%pip install -Uq python_dotenv
!apt-get update
!apt-get install -y poppler-utils

In [ ]:
#Need for Colab
!pip install -q --upgrade "Pillow<11"

In [ ]:
#For Local Run
from dotenv import load_dotenv
load_dotenv()

In [ ]:
#For Colab Run
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from unstructured.partition.pdf import partition_pdf
import os

path = os.path.join(os.getcwd(), "drive/MyDrive/Colab Notebooks/RAG/RRMenu.pdf")

chunks = partition_pdf(
    filename=path,                                                              # only required parameter
    infer_table_structure=False,                                                 # extract tables
    strategy="auto",                                                            # mandatory to infer tables, "high-res" if you extract tables
    languages=["eng"],
    skip_infer_table_types=True,
    # extract_image_block_types=["Image"],                                      # add 'Table' to list to extrac image of table
    # extract_image_block_output_dir= "path",                                   # if none, images and tables will be saved in base64

    # chunking groups related information | Useful for RAG
    chunking_strategy="by_title",                                               # or "basic" | Menu has clear titles
    max_characters=10000,                                                       # max size of chunk
    combine_text_under_n_chars=2000,                                            # comebine different elements when they are under 2000 characters
    new_after_n_chars=6000                                                      # start new part after 6000 characters
    )

In [ ]:
print(len(chunks))

In [ ]:
chunks[0].to_dict()

In [ ]:
texts = []

#Incase we decide to extract images/tables in the future
for chunk in chunks:
    if "CompositeElement" in str(type((chunk))):
        texts.append(chunk)

In [ ]:
from langchain_core.documents import Document


# Chunks --> Langchain Documents
documents = []
for chunk in texts:
  documents.append(Document(page_content = chunk.text,
                            metadata=chunk.metadata.to_dict()                   # Document() expects metadata as a dict, not unstrucuted metadata
                            )
  )

# print(documents[0])

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain_community.vectorstores import Chroma
import shutil, os

filtered_docs = filter_complex_metadata(documents)                              # langchain doesnt like unstructureds metadata structure

# create embeddings for vector storage
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")               # huggingface is free

# remove chroma_db so you dont re-store data
shutil.rmtree("./chroma_db", ignore_errors=True)
os.makedirs("./chroma_db", exist_ok=True)

# Create vectorstore
vectorstore = Chroma.from_documents(filtered_docs,
                                    embeddings,
                                    persist_directory="./chroma_db"
                                    )

In [ ]:
# get a tuple of the top k chunks and their scores based on their similarity score (i.e. cosine sim)
results = vectorstore.similarity_search_with_relevance_scores("Can i make a custom pizza?", k=3)

# print top k docs and their score
for doc, score in results:
    print(f"Score:  {score} \nText: {doc.page_content} \n {'-'*100}")

In [ ]:
import transformers, torch, os
from huggingface_hub import login

login(token=os.getenv("HF_TOKEN"))

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipe = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},                               # half the size of weight
    device_map="auto"                                                           # tries gpu, then cpu
)


In [ ]:
query = "Which burgers are vegetarian?"                                         # user input
context = "\n\n".join(doc.page_content for doc, _ in results[:3])

messages = [
    {"role": "system", "content": "Answer only from the menu below. Be brief."},
    {"role": "user", "content": f"Menu:\n{context}\n\n{query}"}
]

outputs = pipe(
    messages,
    max_new_tokens=150,                                                         # number of tokens to generate
    temperature=0.6,                                                            # more likely to select most probable word (logit / 0.7)
    do_sample=True,                                                             # need to set this to true for temperature to have any effect
    top_p=0.9,                                                                  # model choses from top tokens that make up 90% of the probability mass (ignores super low prob tokens from being selected)
    pad_token_id=pipe.tokenizer.eos_token_id
)

# last element is the llm reply
answer = outputs[0]["generated_text"][-1]["content"]
# print(answer)

In [ ]:
import gradio as gr

def chat_interface(message, history):
    response = answer(message)
    history.append((message, response))
    return history, ""   # clear textbox

css = """

@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600&display=swap');

* {
  font-family: "Inter", sans-serif !important;
}

/* Root background and general text color */
#root {
  background-color: #2e3532;
  color: #e0e2db;
  padding: 18px;
  min-height: 100vh;
}

/* Chat container (box that holds messages) */
#chatbot {
  background-color: #e0e2db;
  color: #2e3532;
  border-radius: 12px;
  padding: 20px;
  max-height: 480px;
  overflow-y: auto;
  margin: 0 auto;
  width: 85%;
  box-shadow: 0px 4px 18px rgba(0,0,0,0.15);
}

/* User message bubble (attempt to match Gradio's generated classes) */
#chatbot .message.user,
#chatbot .chatbot-message.user {
  background: #73BA9B;
  color: white;
  border-radius: 12px 12px 0 12px; /* rounded bottom-left square */
  padding: 10px 14px;
  margin: 6px 0;
  max-width: 80%;
  box-shadow: 0 2px 8px rgba(0,0,0,0.2);
}



#chatbot .message, #chatbot .chatbot-message {
  animation: fadeIn 0.25s ease-in-out;
}

@keyframes fadeIn {
  from { opacity: 0; transform: translateY(6px); }
  to { opacity: 1; transform: translateY(0); }
}



/* Assistant/bot message bubble */
#chatbot .message.bot,
#chatbot .chatbot-message.bot {
  background: #ffffff;
  color: #2e3532;
  border-radius: 12px 12px 12px 0; /* rounded bottom-right square */
  padding: 10px 14px;
  margin: 6px 0;
  max-width: 80%;
  box-shadow: 0 2px 8px rgba(0,0,0,0.15);
}

/* Textbox styling */
#msg textarea {
  background-color: #e0e2db !important;
  color: #2e3532 !important;
  border-radius: 8px !important;
  padding: 8px !important;
  box-shadow: none !important;
}

/* Button styling (Clear and default buttons) */
#clear, .gr-button {
  background-color: #01110A !important;
  color: #e0e2db !important;
  border: none !important;
  box-shadow: none !important;
}

/* Title / markdown text color + CENTERING */
#title {
  color: #e0e2db;
  text-align: center;
  width: 100%;
  display: block;
  margin: 0 auto;
}

/* Ensure Gradio's dark borders don't show */
.gradio-container .panel {
  background: transparent !important;
  box-shadow: none !important;
}

#footer {
  text-align: center;
  color: #01110A;
  margin-top: 30px;
  padding-top: 10px;
  font-size: 14px;
  opacity: 0.8;
}
"""

with gr.Blocks(css=css, elem_id="root") as demo:

    gr.Markdown("## Menu Chatbot", elem_id="title", elem_classes=["title"])


    chatbot = gr.Chatbot(value=[("Assistant", "Hello! How may I assist you today?")], elem_id="chatbot")

    msg = gr.Textbox(
        label="Your question",
        placeholder="Type and press Enter...",
        elem_id="msg"
    )

    examples = gr.Examples(
        examples=[
            "What are some good vegetarian options?",
            "I am not so hungry, what is a good appetizer",
            "If I want to order dessert what are my options?"
        ],
        inputs=[msg]
    )

    clear = gr.Button("Clear", elem_id="clear")

    msg.submit(chat_interface, [msg, chatbot], [chatbot, msg])
    clear.click(lambda: [], None, chatbot)

    gr.Markdown(
        """
        <div id="footer">
            Built for the Resuarant ChatBot RAG Model Project — Penn State · 2025
        </div>
        """,
        elem_id="footer"
    )


with gr.Tabs():
    with gr.Tab("Chat"):
        ...
    with gr.Tab("About"):
        gr.Markdown("""
        ### ℹ About This Project
        This chatbot uses a Retrieval-Augmented Generation (RAG) pipeline
        to answer questions about food waste based on EPA reports.
        """)

demo.launch(show_error=True)